# Data Aggregation & Combination

## Author: Nika Faraji 

### Data Science Capstone

In [1]:
#JH: Just a general note to try and use/define methods where possible, 
#Ideally we would move our methods into some scripts and import them as a package
#This might not make sense if the method is only used in one script so for now I think were okay
#but just something we should keep in mind if we find ourselves using the same code over and over

import pandas as pd
import numpy as np

df_2022 = pd.read_csv("data/mhcld_puf_2022.csv")
df_2021 = pd.read_csv("data/mhcld_puf_2021.csv")
df_2020 = pd.read_csv("data/mhcld_puf_2020.csv")
df_2019 = pd.read_csv("data/mhcld_puf_2019.csv")
df_2018 = pd.read_csv("data/mhcld_puf_2018.csv")
df_2017 = pd.read_csv("data/mhcld_puf_2017.csv")
df_2016 = pd.read_csv("data/mhcld_puf_2016.csv")
df_2015 = pd.read_csv("data/mhcld_puf_2015.csv")
df_2014 = pd.read_csv("data/mhcld_puf_2014.csv")
df_2013 = pd.read_csv("data/mhcld_puf_2013.csv")

In [8]:
pop_df = pd.read_csv("data/nst-est2020.csv")

In [2]:
df_2022 = df_2022.groupby(['STATEFIP', 'YEAR'], as_index=False).sum()
df_2021 = df_2021.groupby(['STATEFIP', 'YEAR'], as_index=False).sum()
df_2020 = df_2020.groupby(['STATEFIP', 'YEAR'], as_index=False).sum()
df_2019 = df_2019.groupby(['STATEFIP', 'YEAR'], as_index=False).sum()
df_2018 = df_2018.groupby(['STATEFIP', 'YEAR'], as_index=False).sum()
df_2017 = df_2017.groupby(['STATEFIP', 'YEAR'], as_index=False).sum()
df_2016 = df_2016.groupby(['STATEFIP', 'YEAR'], as_index=False).sum()
df_2015 = df_2015.groupby(['STATEFIP', 'YEAR'], as_index=False).sum()
df_2014 = df_2014.groupby(['STATEFIP', 'YEAR'], as_index=False).sum()
df_2013 = df_2013.groupby(['STATEFIP', 'YEAR'], as_index=False).sum()


merged_df = pd.concat([df_2022, df_2021, df_2020, df_2019, df_2018, df_2017, df_2016, df_2015, df_2014, df_2013], 
                      ignore_index=True)

In [ ]:
merged_df

In [ ]:
df = merged_df[['STATEFIP', 'YEAR', 'ANXIETYFLG', 'TRAUSTREFLG', 'ADHDFLG', 'BIPOLARFLG', 'DEPRESSFLG',
        'SPHSERVICE', 'CMPSERVICE', 'OPISERVICE', 'RTCSERVICE']]

df['TOT_SERVICE'] = df[['SPHSERVICE', 'CMPSERVICE', 'OPISERVICE', 'RTCSERVICE']].sum(axis=1)

df

In [ ]:
state_code_to_name = {
    1: 'AL', 2: 'AK', 4: 'AZ', 5: 'AR', 6: 'CA', 8: 'CO', 9: 'CT', 10: 'DE', 11: 'DC',
    12: 'FL', 13: 'GA', 15: 'HI', 16: 'ID', 17: 'IL', 18: 'IN', 19: 'IA', 20: 'KS',
    21: 'KY', 22: 'LA', 25: 'MA', 26: 'MI', 27: 'MN', 28: 'MS',
    29: 'MO', 30: 'MT', 31: 'NE', 32: 'NV', 34: 'NJ', 35: 'NM', 36: 'NY',
    37: 'NC', 38: 'ND', 39: 'OH', 40: 'OK', 41: 'OR', 42: 'PA', 44: 'RI', 45: 'SC',
    46: 'SD', 47: 'TN', 48: 'TX', 49: 'UT', 50: 'VT', 51: 'VA', 53: 'WA',
    55: 'WI', 56: 'WY', 72: 'PR', 99: 'Other'
}

# Map the state codes to state names
df['STATE'] = df['STATEFIP'].map(state_code_to_name)
df

In [ ]:
# Reshape the DataFrame using pd.melt()
df_long = pd.melt(pop_df, id_vars=['STATE', 'NAME'], var_name='YEAR', value_name='POPESTIMATE')

# Clean up the 'Year' column by extracting the year number from the column names
df_long['YEAR'] = df_long['YEAR'].str.extract('(\d{4})')
df_long['YEAR'] = df_long['YEAR'].astype(int)

df_long

In [ ]:
data = pd.merge(df, df_long, left_on=['STATEFIP', 'YEAR'], right_on=['STATE', 'YEAR'], how='inner')
data = data.drop('STATE_y', axis=1)
data['UTIL_RATE'] = data['TOT_SERVICE']/data['POPESTIMATE']
data

In [ ]:
trend_df = pd.read_csv("data/pytrends_data.csv")

# Split 'key' column into 'state' and 'year' columns
trend_df[['country_state', 'year']] = trend_df['key'].str.split('_', expand=True)
trend_df['state'] = trend_df['country_state'].str[-2:]  # Extract the state abbreviation from the country_state part
trend_df['year'] = trend_df['year'].astype(int)  # Convert year to integer
trend_df = trend_df.drop(['key', 'country_state', 'Unnamed: 0'], axis=1) #remove uneeded columns

trend_df

In [ ]:
#merge health data & trend data on year & state
data_1 = pd.merge(data, trend_df, left_on=['STATE_x', 'YEAR'], right_on=['state', 'year'], how='inner')
data_1 = data_1.drop('year', axis=1) #remove uneeded columns
data_1

In [13]:
data_1.to_csv('data/aggregated_data.csv', index=False)